In [ ]:
import os
import sys
sys.path.append('/workspace/pring/fine-tune/server/Async-Python-Server')
os.chdir('/workspace/pring/fine-tune/');

print(os.getcwd())
os.chdir('/workspace/pring/fine-tune/server/Async-Python-Server')

from api.llm.metapython import *

import pandas
#!rm -rf /workspace/pring/llama-2-7b-chat-hf1*
#!pip install -U transformers
#!conda install --yes tokenizers
#!pip install --upgrade pip
#!pip install tokenizers==0.15.2
#!git clone https://github.com/NVIDIA/apex
#!pip install -v --disable-pip-version-check --no-cache-dir --no-build-isolation --config-settings "--build-option=--cpp_ext" --config-settings "--build-option=--cuda_ext" ./apex/
import transformers
#!pip install transformers==4.13.0
!conda install --yes pytorch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 pytorch-cuda=11.8 -c pytorch -c nvidia
#!conda install --yes transformers[deepspeed]

try:
  import torch
except:
  !conda install --yes accelerate peft bitsandbytes transformers trl dataset torch==2.1.1
  !conda install --yes pytorch==2.1.1
import torch
if (torch.cuda.is_available() == False ):
  !conda install --yes torch==2.1.1
  !conda install --yes accelerate peft bitsandbytes transformers trl dataset torch==2.1.1
  import torch
if (torch.cuda.is_available() == False ):
    raise Exception("Reinstall pytorch")
    
import os
import sys
sys.path.append('/workspace/pring/fine-tune/server/Async-Python-Server')
os.chdir('/workspace/pring/fine-tune/');
import torch
import transformers
print(os.getcwd())
os.chdir('/workspace/pring/fine-tune/server/Async-Python-Server');
print(os.getcwd())
import pandas
from urllib.parse import quote
from urllib.parse import unquote
import time
!conda install --yes sentencepiece
import sentencepiece
import tensorboardX

import gc
torch.cuda.empty_cache()
gc.collect()
gc.collect()

magic('nvidia-smi')

import sys
fromImport("datasets",["load_dataset"])
fromImport("transformers",["AutoModelForCausalLM", "AutoTokenizer","BitsAndBytesConfig","HfArgumentParser","TrainingArguments","pipeline","TextStreamer","TextIteratorStreamer","logging"])
fromImport("peft",["LoraConfig", "PeftModel"])
from trl import SFTTrainer
import time
importLib("datasets")
importLib("trl")
importLib("accelerate")
from threading import Thread
#from api.xpy import *

#model_name = "Weblet/llama-2-usaaef2c"
model_name = "NousResearch/Llama-2-7b-chat-hf"


################################################################################
# QLoRA parameters
################################################################################
lora_r = 64 # LoRA attention dimension
lora_alpha = 16 # Alpha parameter for LoRA scaling
lora_dropout = 0.1 # Dropout probability for LoRA layers

################################################################################
# bitsandbytes parameters
################################################################################
use_4bit = True # Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16" # Compute dtype for 4-bit base models
bnb_4bit_quant_type = "nf4" # Quantization type (fp4 or nf4)
use_nested_quant = False # Activate nested quantization for 4-bit base models (double quantization)

################################################################################
# TrainingArguments parameters
################################################################################
output_dir = "./results" # Output directory where the model predictions and checkpoints will be stored
num_train_epochs = 1 # Number of training epochs
fp16 = False # Enable fp16/bf16 training (set bf16 to True with an A100)
bf16 = False
per_device_train_batch_size = 1 # Batch size per GPU for training
per_device_eval_batch_size = 1 # Batch size per GPU for evaluation
gradient_accumulation_steps = 1 # Number of update steps to accumulate the gradients for
gradient_checkpointing = True # Enable gradient checkpointing
max_grad_norm = 0.3 # Maximum gradient normal (gradient clipping)
learning_rate = 2e-4 # Initial learning rate (AdamW optimizer)
weight_decay = 0.001 # Weight decay to apply to all layers except bias/LayerNorm weights
optim = "paged_adamw_32bit" #"adamw_apex_fused"# Optimizer to use
lr_scheduler_type = "cosine" # Learning rate schedule
max_steps = -1 # Number of training steps (overrides num_train_epochs)
warmup_ratio = 0.03 # Ratio of steps for a linear warmup (from 0 to learning rate)
group_by_length = False # Group sequences into batches with same length # Saves memory and speeds up training considerably
save_steps = 0 # Save checkpoint every X updates steps
logging_steps = 25 # Log every X updates steps

################################################################################
# SFT parameters
################################################################################
max_seq_length = 512 # Maximum sequence length to use
packing = False # Pack multiple short examples in the same input sequence to increase efficiency
device_map = {"": 0} #  Load the entire model on the GPU 0
import gc
gc.collect()
gc.collect()

import locale
locale.getpreferredencoding = lambda: "UTF-8"
print("marker")


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )
except:
    magic('%pip install -U bitsandbytes');
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )
print(use_4bit)
# Check GPU compatibility with bfloat16
try:
  if compute_dtype == torch.float16 and use_4bit:
      major, _ = torch.cuda.get_device_capability()
      if major >= 8:
          print("=" * 80)
          print("Your GPU supports bfloat16: accelerate training with bf16=True")
          print("=" * 80)
except:
  device_map = "cpu"

print(device_map)
# Load base model
try:
  model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,device_map=device_map,trust_remote_code=True)
except:
  model = AutoModelForCausalLM.from_pretrained(model_name,device_map=device_map,trust_remote_code=True)
model.config.use_cache = True
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 

logging.set_verbosity(logging.CRITICAL)

streamer = TextStreamer(tokenizer, skip_prompt=True)
from api.xpy import *
def streamWriter(str):
    txt=''
    transaction_id=''
    for s in str:
        txt+=tokenizer.decode(s)
        if 'ESNOPSER_ID' in txt:
            transaction_id=txt.split('ESNOPSER_ID')[1].split('<')[0].split('[')[0];
            transaction_id
   # print(txt)
    stream=streamWriter.responseStream
    if at(streamWriter.requestMap,[transaction_id]) != None:
        stream = at(streamWriter.requestMap,[transaction_id])
    if stream != None:
        zwrite_response_body(streamWriter.responseStream, b(f"{txt} "))
streamWriter.responseStream=None
streamWriter.requestMap={}
streamer.put=streamWriter
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_seq_length,streamer=streamer)

prompt_seed = ""# """You are a chatbot with the purpose of serving USAA members. Your goal is to give sound financial advice to USAA members and to direct USAA members where to find additional resources to guide them through their financial journey."""


def talk(prompt):
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    return result[0]['generated_text']
async def talks(prompt,transaction_id=''):
    return pipe(f"<pre>{prompt_seed}</pre>{transaction_id}<s>[INST] I am curious. {prompt} [/INST]")



print(talk(prompt_seed))

#os.chdir('/workspace/pring/fine-tune/server/Async-Python-Server');

from http.server import ThreadingHTTPServer, BaseHTTPRequestHandler
import http.client
import asyncio
import time
from api.xpy import *
import socket
import platform
from http.server import ThreadingHTTPServer, BaseHTTPRequestHandler
import asyncio
from api.promises import *
from api.excepts import *
from api.xhttp import *
from api.zfile import *
from api.xpy import *
import mimetypes
mimetypes.init()

#os.chdir('/workspace/pring/fine-tune/');

class handler(BaseHTTPRequestHandler):

  async def do_ASYNC(request, data):
      print(f"Recieved {request.path}")
      if(request.path.startswith('/llama')):
          transaction_id=f"ESNOPSER_ID{time.time()}".replace('.','');
          streamWriter.responseStream=request
          streamWriter.requestMap[transaction_id]=request;
          await zsendHeaders(request,{
              'status':200,
              'Content-type':'text/plain',
              'Access-Control-Allow-Origin':'*'
          })
          content=await talks(unquote(f"{at(request.path.split('/llama?'),[1])}"),transaction_id=transaction_id)
          text=f"[FULLTEXT]{content[0]['generated_text']}"
          print(f'responding{text}')
          await zwriteResponseBody(request, b(text))
          del streamWriter.requestMap[transaction_id]
      else:
          path = f"/workspace/pring/fine-tune/server/Async-Python-Server{request.path}".split("?")[0].split("#")[0];
          contentType = mimetypes.guess_type(path,strict=False)
          print(path,contentType[0])
          content = await zreadFileBytes(path)
          await zsendHeaders(request,{'status':200,'Content-type':contentType[0]})
          await zwriteResponseBody(request, content)

  def do_METHOD(request):
    asyncio.run(request.do_ASYNC(request))

  do_GET = do_METHOD
  do_OPTIONS = do_METHOD
  do_POST = do_METHOD
  do_PUT = do_METHOD
  do_PATCH = do_METHOD
  do_HEAD = do_METHOD
  do_DELETE = do_METHOD
  do_CONNECT = do_METHOD
  do_TRACE = do_METHOD

println("\n\nPlatform Node: ",platform.node())
println("Version Info: ",sys.version_info)
println("Hostname: ",socket.gethostname())
println("FQDN: ",socket.getfqdn())

httpd = ThreadingHTTPServer(('', 80), handler)
httpd.timeout = 30
httpd.serve_forever()

/workspace/pring/fine-tune


/workspace/pring/fine-tune/server/Async-Python-Server/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/pytorch

  added / updated specs:
    - pytorch-cuda=11.8
    - pytorch==2.1.1
    - torchaudio==2.1.1
    - torchvision==0.16.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-python-1.0.9        |  py311h6a678d5_8         358 KB
    ca-certificates-2024.9.24  |       h06a4308_0         130 KB
    certifi-2024.8.30          |  py311h06a4308_0         163 KB
    charset-normalizer-3.3.2   |     pyhd3eb1b0_0          44 KB
    ffmpeg-4.3                 |       hf484d3e_0         9.9 MB  pytorch
    gnutls-3.6.15              |       he1e5248_0         1.0 MB
    idna-3.7                   |  py311h06a4308_0         133 KB
    jpeg-9e                    |       h5eee18b_3         262 KB
    lame-

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.59s/it]



<s>[INST]  [/INST]  I'm not sure I understand what you are saying with "]. Could you explain?
 everybody loves me."? Could you provide more context or clarify your message?


Platform Node:  patrick-api-0-0 

Version Info:  sys.version_info(major=3, minor=11, micro=5, releaselevel='final', serial=0) 

Hostname:  patrick-api-0-0 

FQDN:  patrick-api-0-0 

Recieved /
/workspace/pring/fine-tune/server/Async-Python-Server/ None
Recieved /favicon.ico
/workspace/pring/fine-tune/server/Async-Python-Server/favicon.ico image/vnd.microsoft.icon


...


responding[FULLTEXT]<pre></pre>ESNOPSER_ID1731604066493389<s>[INST] I am curious. Who are you? [/INST]  I'm just an AI assistant developed by Meta AI to help users like you with their queries. I'm not a human, but a machine learning model trained to understand and respond to natural language inputs. My purpose is to assist and provide helpful responses to your questions to the best of my abilities. Is there anything else I can help you with?
Recieved /llama?hi


10.244.11.0 - - [14/Nov/2024 17:45:03] "GET /llama?hi HTTP/1.1" 200 -



responding[FULLTEXT]<pre></pre>ESNOPSER_ID1731606303615987<s>[INST] I am curious. hi [/INST]  I'm just an AI, I don't have personal experiences or feelings, so I can't feel curious or any other emotions. However, I'm here to help you with any questions or tasks you may have! Is there something specific you would like to know or discuss?
Recieved /llama?hi


10.244.11.0 - - [14/Nov/2024 18:10:16] "GET /llama?hi HTTP/1.1" 200 -



responding[FULLTEXT]<pre></pre>ESNOPSER_ID17316078166053517<s>[INST] I am curious. hi [/INST]  I'm just an AI, I don't have personal experiences or feelings, so I can't feel curious or any other emotions. However, I'm here to help you with any questions or tasks you may have, so feel free to ask me anything!
